## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("..\Vacation_Search\WeatherPy_vacation.csv")
vacation_df.head()

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,5,Grand Gaube,MU,77.38,clear sky,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
1,8,Port-Gentil,GA,80.56,broken clouds,-0.7193,8.7815,Fengshui Residence
2,10,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
3,13,Lorengau,PG,83.64,broken clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
4,17,Faya,SA,77.52,broken clouds,18.3851,42.4509,مقبرة ال مجمل


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""
# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

In [4]:
vacation_df.columns

Index(['Unnamed: 0', 'City', 'Country', 'Max Temp', 'Current Description',
       'Lat', 'Lng', 'Hotel Name'],
      dtype='object')

In [5]:
# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations_df = vacation_df[["City","Lat","Lng"]]
locations_df

,City,Lat,Lng
0,Grand Gaube,-20.0064,57.6608
1,Port-Gentil,-0.7193,8.7815
2,Vaini,-21.2000,-175.2000
3,Lorengau,-2.0226,147.2712
4,Faya,18.3851,42.4509
...,...,...,...
212,Kutum,14.2000,24.6667
213,Tarakan,3.3000,117.6333
214,Baghdad,33.3406,44.4009
215,Sabang,5.8933,95.3214


In [6]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [11]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['City']=='Cabo San Lucas']
vacation_end = vacation_df.loc[vacation_df['City']=='Cabo San Lucas']
vacation_stop1 = vacation_df.loc[vacation_df['City']=='Puerto Escondido']
vacation_stop2 = vacation_df.loc[vacation_df['City']=='Lazaro Cardenas']
vacation_stop3 = vacation_df.loc[vacation_df['City']=='Jalpa']

In [12]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[["Lat","Lng"]].to_numpy()[0]
end = vacation_end[["Lat","Lng"]].to_numpy()[0]
stop1 = vacation_stop1[["Lat","Lng"]].to_numpy()[0]
stop2 = vacation_stop2[["Lat","Lng"]].to_numpy()[0]
stop3 = vacation_stop3[["Lat","Lng"]].to_numpy()[0]
start

array([  22.8909, -109.9124])

In [13]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
itinerary = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3],
                                  travel_mode='DRIVING')
fig.add_layer(itinerary)
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,146,Cabo San Lucas,MX,86.81,broken clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
1,183,Puerto Escondido,MX,78.91,overcast clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
2,521,Lazaro Cardenas,MX,80.49,overcast clouds,17.9583,-102.2000,quinta antigua
3,227,Jalpa,MX,75.00,overcast clouds,21.6333,-102.9667,"Estacionamiento ""El Nogal"""


In [18]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt></dd>{Current Description}</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
itinerary_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
itin_locations = itinerary_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(35, 136.0), zoom_level=5)
marker_layer = gmaps.marker_layer(itin_locations, info_box_content=itinerary_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))